In [ ]:
# 角速度の平面成分と加速度の平面成分を用いてZ軸を基準に原点で回転させることで歩行を補正する

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib_fontja
import numpy as np
from scipy import signal

acce_file_path = './PDR_1/Accelerometer.csv'
gyro_file_path = './PDR_1/Gyroscope.csv'

acce_data = pd.read_csv(acce_file_path,names=['timestamp','x','y','z'], header=0)
gyro_data = pd.read_csv(gyro_file_path,names=['timestamp','x','y','z'], header=0)

# 2sから9sまでのデータに絞る
acce_data = acce_data[(acce_data['timestamp'] >= 2.0) & (acce_data['timestamp'] <= 9.0)]
gyro_data = gyro_data[(gyro_data['timestamp'] >= 2.0) & (gyro_data['timestamp'] <= 9.0)]

In [ ]:
# 加速度と角速度のグラフを横に並べて表示
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 加速度
axes[0].plot(acce_data['timestamp'], acce_data['x'], label='X')
axes[0].plot(acce_data['timestamp'], acce_data['y'], label='Y')
axes[0].plot(acce_data['timestamp'], acce_data['z'], label='Z')
axes[0].set_xlabel('Time [s]')
axes[0].set_ylabel('Acceleration [m/s^2]')
axes[0].set_title('Accelerometer Data')
axes[0].legend()
axes[0].grid()

# 角速度
axes[1].plot(gyro_data['timestamp'], gyro_data['x'], label='X')
axes[1].plot(gyro_data['timestamp'], gyro_data['y'], label='Y')
axes[1].plot(gyro_data['timestamp'], gyro_data['z'], label='Z')
axes[1].set_xlabel('Time [s]')
axes[1].set_ylabel('Angular Velocity [deg/s]')
axes[1].set_title('Gyroscope Data')
axes[1].legend()
axes[1].grid()

plt.tight_layout()
plt.show()

## 加速度

X軸は重力加速度方向を向いている
平面加速度成分は(Y,Z)

## 角速度

加速度の値より、角速度の平面成分はX軸である


In [ ]:
# ノルムの算出
acce_data['norm'] = np.sqrt(acce_data['y']**2 + acce_data['z']**2)
acce_data['low_norm'] = acce_data['norm'].rolling(window=10).mean()

peak,_ = signal.find_peaks(acce_data['low_norm'], height=4, distance=20)
gensoku_index = [303, 352, 413, 478, 554]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 加速度
axes[0].plot(acce_data['timestamp'], acce_data['y'], label='Y')
axes[0].plot(acce_data['timestamp'], acce_data['z'], label='Z')
axes[0].set_xlabel('Time [s]')
axes[0].set_ylabel('Acceleration [m/s^2]')
axes[0].set_title('Accelerometer Data')
axes[0].legend()
axes[0].grid()

# ノルム
axes[1].plot(acce_data['timestamp'], acce_data['norm'], label='Norm', color='black', alpha=0.3)
axes[1].plot(acce_data['timestamp'], acce_data['low_norm'], label='Low-pass Norm', color='green')
axes[1].plot(acce_data['timestamp'].iloc[peak], acce_data['low_norm'].iloc[peak], 'x', label='Peaks', color='blue')
axes[1].plot(acce_data['timestamp'].iloc[gensoku_index], acce_data['low_norm'].iloc[gensoku_index], 'o', label='Reference', color='orange')
axes[1].axhline(y=3.0, color='red', linestyle='--', label='Gravity (9.81 m/s²)')
axes[1].set_xlabel('Time [s]')
axes[1].set_ylabel('Acceleration [m/s^2]')
axes[1].set_title('Acceleration Norm')
axes[1].legend()
axes[1].grid()

plt.tight_layout()
plt.show()

In [ ]:
# 角速度の平面成分を積分して角度を求める
gyro_data['angle'] = np.cumsum(gyro_data['x']) / (len(gyro_data) / (gyro_data['timestamp'].iloc[-1] - gyro_data['timestamp'].iloc[0]))
gyro_data['low_angle'] = gyro_data['angle'].rolling(window=60, center=True).mean()

plt.plot(gyro_data['timestamp'], gyro_data['angle'], label='Angle', color='blue', alpha=0.3)
plt.plot(gyro_data['timestamp'], gyro_data['low_angle'], label='Low-pass Angle', color='orange')
plt.xlabel('Time [s]')
plt.ylabel('Angle [deg]')
plt.title('Integrated Angle from Gyroscope')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# 一歩ずつの加速度の方向をプロットする

fig, axes = plt.subplots(1,5,figsize=(18, 6))
for i in range(5):
	second_i = gensoku_index[i]
	axes[i].quiver(acce_data['y'].iloc[i], acce_data['z'].iloc[i], acce_data['y'].iloc[second_i] - acce_data['y'].iloc[i], acce_data['z'].iloc[second_i] - acce_data['z'].iloc[i], angles='xy', scale_units='xy', scale=1, color='red')
	axes[i].vlines(acce_data['y'].iloc[second_i], 0, 5, colors='blue', linestyles='dashed', label='Y=1')
	# z軸を基準に回転させる

	axes[i].set_xlim(-2.5, 2.5)
	axes[i].set_ylim(0, 5)
	axes[i].set_xlabel('Y Acceleration [m/s^2]')
	axes[i].set_ylabel('Z Acceleration [m/s^2]')
	axes[i].set_title(f'Step {i+1}')
	axes[i].grid()
	axes[i].set_aspect('equal', adjustable='box')
plt.tight_layout()
plt.show()

plt.plot(gyro_data['timestamp'], gyro_data['low_angle'], label='Low-pass Angle', color='orange')
for i in range(len(peak)):
	plt.scatter(gyro_data['timestamp'].iloc[peak[i]], gyro_data['low_angle'].iloc[peak[i]], color='blue', linestyle='dashed')
	print(f"Peak {i+1}: deg{gyro_data['low_angle'].iloc[peak[i]]},rad{np.deg2rad(gyro_data['low_angle'].iloc[peak[i]])}")
plt.xlabel('Time [s]')
plt.ylabel('Angle [deg]')
plt.title('Integrated Angle from Gyroscope')
plt.legend()
plt.grid()
plt.show()

In [ ]:
step = 0.3
point = [[0, 0]]

# 
for p in peak:
    x = step * np.cos(gyro_data['low_angle'][p]*1.2) + point[-1][0]
    y = step * np.sin(gyro_data['low_angle'][p]*1.2) + point[-1][1]

    point.append([x, y])

point = pd.DataFrame(data=point, columns=['x', 'y'])

point['x'] = point['x'] - point['x'][0]
point['y'] = point['y'] - point['y'][0]

# 描画
size = ((-1,5), (-1, 2))

plt.plot(point['x'], point['y'], '.-', label='推定', zorder=1)

# 軸を揃える
plt.gca().set_aspect('equal', adjustable='box')

plt.title('PDRのみの歩行軌跡')
plt.xlabel('x[m]')
plt.ylabel('y[m]')
plt.xticks(np.arange(size[0][0], size[0][1], 0.3))
plt.yticks(np.arange(size[1][0], size[1][1], 0.3))
plt.hlines(0, size[0][0], size[0][1], colors='gray', linestyles='--', zorder=0)
plt.xlim(size[0][0], size[0][1])
plt.ylim(size[1][0], size[1][1])

plt.grid()
plt.show()